In [1]:
## extracts features from audio files and converts into numpy
import librosa
import numpy as np
import os, re, csv, sys
from datetime import datetime

global hop_length, mfcc_len, chrom_len

# Set the hop length; at 22050 Hz, 512 samples ~= 23ms
hop_length = 128


In [11]:
## return a (flatten) one-D array of mfcc of an audio file
def getFlattenMFCC(audio_file):
    try:
        y, sr = librosa.load(audio_file)
    except FileNotFoundError:
        print('No such file or directory')

    # Compute MFCC features from the raw signal
    print(audio_file)
    return librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13).flatten()

## return a (flatten) one-D array of chromagram of an audio file
def getChroma(audio_file):

    y, sr = librosa.load(audio_file)
    # Separate harmonics and percussives into two waveforms
    y_harmonic, y_percussive = librosa.effects.hpss(y)

    # Compute chroma features from the harmonic signal
    return librosa.feature.chroma_cqt(y=y_harmonic,
                                            sr=sr).flatten()


## return a list of 1-d array of chromagram padded with 0's of ALL audio files
def getPaddedChroma(audio_files):
    result = [getChroma(f) for f in audio_files]

    ##pad arrays with 0's. Get arrays of size Max
    max_len = max([len(x) for x in result])
    padded = [np.pad(x, (0, max_len - len(x)), mode = 'constant') for x in result]

    ##sanity check
    is_shorter = sum([len(x) - max_len for x in padded])
    if is_shorter < 0:
        print('not padded well')
        return -1
    else:
        return padded

In [8]:
## return a list of 1-d array of MFCC padded with 0's of ALL audio files
def getPaddedMFCC(audio_files, train = True):
    
    result = [getFlattenMFCC(f) for f in audio_files]

    if train:
        ##pad arrays with 0's. Get arrays of size Max
        max_len = max([len(x) for x in result])
    else:
        max_len = mfcc_len
        
    padded = [np.pad(x, (0, max_len - len(x)), mode = 'constant') for x in result]

    ##sanity check
    is_shorter = sum([len(x) - max_len for x in padded])
    if is_shorter < 0:
        print('not padded well')
        return -1
    else:
        return padded, max_len

In [9]:
def process_train_audio(file_list):
    
    print('Processing ', len(file_list), ' files')
    
    ####MFCC
    mfcc = [getFlattenMFCC(f) for f in file_list]
    padded_mdcc, mfcc_len = getPaddedMFCC(mfcc)
    chrom = [getFlattenChrom(f) for f in file_list]
    padded_chrom, chrome_len = getPaddedChrom(chrom)
    
    
    
    ### user cannot fix this
    assert (mfcc != -1 and chrom != -1), "Audio process does not produce uniform format."

    
    
    ## concatenate mfcc and chrom features
    attr_input = [np.hstack([m, c]) for m, c in zip(padded_mdcc, padded_chrom)]
    ## checking the final length
    #print(len(mfcc[5]), len(chrom[5]), len(x[5]))


    ##detect targets from sound names
    p = re.compile('^[aeou]|[bcdfghjklmnpqrstwxyz]+(?=[aeiou])')
    target_input = [p.match(f).group() for f in audio_files]
    
    print('Processing finished')
    return attr_input, target_input

##to process audio file for both validation and testing
def process_validate_audio(file_list):
    
    print('Processing ', len(file_list), ' files')
    
    ####MFCC
    mfcc = [getFlattenMFCC(f) for f in file_list]
    padded_mdcc = getPaddedMFCC(mfcc)
    chrom = [getFlattenChrom(f) for f in file_list]
    padded_chrom = getPaddedChrom(chrom)
    
    ### user cannot fix this
    assert (mfcc != -1 and chrom != -1), "Audio process does not produce uniform format."

    
    
    ## concatenate mfcc and chrom features
    attr_input = [np.hstack([m, c]) for m, c in zip(padded_mdcc, padded_chrom)]
    ## checking the final length
    #print(len(mfcc[5]), len(chrom[5]), len(x[5]))


    ##detect targets from sound names
    p = re.compile('^[aeou]|[bcdfghjklmnpqrstwxyz]+(?=[aeiou])')
    target_input = [p.match(f).group() for f in audio_files]
    
    print('Processing finished')
    return attr_input, target_input

def writeCSV(file, file_name):
    
    with open(file_name,"w+") as processed:
            csvWriter = csv.writer(processed,delimiter=',')
            csvWriter.writerows(file)
    print('File saved at ', file_name)        
    
    
    
    

In [12]:
import os
audio_dir = '/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/'
all_files = [audio_dir + d for d in os.listdir(audio_dir)]

att, tar = process_train_audio(all_files)
writeCSV(att, 'train_attr.csv')
writeCSV(tar, 'train_tar.csv')

Processing  784  files
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/seng1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cui3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/shu4_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xiong2_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/sou4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xuan4_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cuan2_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cun3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qu1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cen1_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/si1_FV3_MP3.mp3
/Users/panchanok/

/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/sang2_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chang4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cu1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cao1_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/su3_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cha3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xing3_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qi3_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xie4_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xun4_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chou4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHac

/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/suan3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/que4_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chun4_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chi1_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/quan4_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/shan1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/si2_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xue1_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/song1_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xin1_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/she3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyH

/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qing3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cu2_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/sui4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/sang1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/ca1_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xiang1_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qun2_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/san3_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cang3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qie2_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/se4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHac

/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/shei4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chan3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/sha4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xu4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chuang3_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xiong1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/seng2_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cheng4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cong3_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chai3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qu2_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack20

/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chu1_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/sa1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qing1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/ca3_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/sang3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/sen4_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xia2_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qiao1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cou1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chuo3_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xiu1_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHac

/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chuang4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/shuai2_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/shui1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cheng3_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cong4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chai1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cun2_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/shun4_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cuan3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chan1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xiong3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyH

/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/shuang3_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cuo4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/shuo2_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qi1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/sai2_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xing1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qie3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cang2_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/san2_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/xiao1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qun3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/P

/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qiong1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chuan1_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qian2_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/cai3_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chen2_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/chuai4_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/ci1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/ceng1_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/she2_FV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/si3_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/qiong4_MV3_MP3.mp3
/Users/panchanok/Desktop/PyHack201

TypeError: Invalid file: array([-261.31927 , -247.174   , -221.6634  , ...,  -21.35182 ,
        -19.574238,  -17.349373], dtype=float32)

If there is error from the above chunk, make sure only mp3 is included in aggr/. Check even invisible .dstore

In [ ]:
validate_path = '/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/validate/'
#process_audio(validate_path, train = True)

In [ ]:
test_path = '/Users/panchanok/Desktop/PyHack2019/PyHack2019/sound_samples/uservoice/'
process_audio(test_path, train = False)

Try displaying audio

In [ ]:
# Displays are built with matplotlib 
import matplotlib.pyplot as plt
import librosa

# Let's make plots pretty
import matplotlib.style as ms
ms.use('seaborn-muted')

# Render figures interactively in the notebook
%matplotlib nbagg

# IPython gives us an audio widget for playback
from IPython.display import Audio

In [ ]:

plt.figure()
librosa.display.waveplot(y=y, sr=sr)
plt.figure()
librosa.display.specshow(melspec, y_axis='mel', x_axis='time')
plt.colorbar()